# 1. \[40pts\] Implement the various functions for common Elasticsearch operations

In [ ]:
!pip install pyyaml elasticsearch elasticsearch_dsl

In [ ]:
# load packages
from src.elasticsearch_client import ElasticsearchClient
from data.index_mapping import mapping
import json

INDEX_NAME = "movie_reviews_homework8"

In [ ]:
# Connect to Elasticsearch
es_client = ElasticsearchClient()
print(es_client.client.info())

In [ ]:
# Delete index if exists before creating a new one
try:
    ###### EDIT HERE ######
    pass
    ###### EDIT HERE ######
except Exception as e:
    print(f"Error {str(e)}")

In [ ]:
# create index based on mapping
###### EDIT HERE ######

###### EDIT HERE ######

In [ ]:
# load data
with open("./data/movie_data_big.json", "r") as f:
    movie_data = json.load(f)

In [ ]:
# insert only one document with doc_id == 0
doc_id = 0
###### EDIT HERE ######

###### EDIT HERE ######

In [ ]:
# delete document with doc_id == 0
doc_id = 0
###### EDIT HERE ######

###### EDIT HERE ######

In [ ]:
# insert all documents using bulk indexing
actions = []
for id_doc, doc in enumerate(movie_data):
###### EDIT HERE ######
    pass




###### EDIT HERE ######

print(len(actions)) # should be 100789

In [ ]:
# get document with doc_id == 85453
doc_id = 85453
###### EDIT HERE ######

###### EDIT HERE ######
print(f"document with id 85453: {doc}")

In [ ]:
# get count of all documents in the index
count = 0
###### EDIT HERE ######

###### EDIT HERE ######
print(count) # should be 100789

In [ ]:
# search for documents with title containing "star wars"
query = {
###### EDIT HERE ######
}
results = es_client.search(index_name=INDEX_NAME, query=query)
###### EDIT HERE ######
for res in results: # should be 10 documents
    print(f"ID: {res['_id']}, Title: {res['_source']['title']}, userID: {res['_source']['userId']}")

# 2. \[10pts\] Compare the difference in execution time between basic and helper operations and report the result and your findings.

In [ ]:
import time
from tqdm import tqdm

In [ ]:
def delete_and_create_index(es_client, index_name, mapping):
    try:
        es_client.delete_index(index_name=index_name)
        es_client.create_index(index_name=index_name, mapping=mapping)
    except Exception as e:
        print(f"Error {str(e)}")

In [ ]:
delete_and_create_index(es_client, INDEX_NAME, mapping)

In [ ]:
# insert all documents not using bulk
start = time.time()
for id_doc, doc in tqdm(enumerate(movie_data)):
    es_client.insert_one_document(index_name=INDEX_NAME, body=doc, doc_id=id_doc)
end = time.time()
print(f"Inserting all documents without bulk took {end - start} seconds")

In [ ]:
# insert all documents using bulk
delete_and_create_index(es_client, INDEX_NAME, mapping)

start = time.time()
actions = []
for id_doc, doc in enumerate(movie_data):
###### EDIT HERE ######
    pass





###### EDIT HERE ######

end = time.time()
print(f"Inserting all documents with bulk took {end - start} seconds")

In [ ]:
from elasticsearch import helpers

In [ ]:
query1 = {
    "query": {
        "match_all": {}
    },
    "size": 1
}

query2 = {
    "query": {
        "bool": {
            "must": [
                {"match": {"genres": "Action"}},
                {"match": {"genres": "Crime"}}
            ]
        }
    },
    "size": 1
}

In [ ]:
page_size=[10, 1000]

# retrieve all documents with scan
for query in [query1, query2]:
    for size in page_size:
        scroll = es_client.scan_index(query=query, index_name=INDEX_NAME, size=size)

        start = time.time()
        count = 0
        for hit in scroll:
            count += 1
        end = time.time()
        print(f"Retrieving all documents with {size} took {end - start} seconds")

In [ ]:
# retrieve all documents with search and scroll

for query in [query1, query2]:
    start = time.time()
    count = 0
    
    response = es_client.client.search(index=INDEX_NAME, body=query)
    scroll_response = es_client.client.search(index=INDEX_NAME, body=query, scroll='1m')
    
    scroll_id = scroll_response['_scroll_id']
    for hit in scroll_response['hits']['hits']:
        count += 1

    while True:
        scroll_response = es_client.client.scroll(scroll_id=scroll_id, scroll='1m')
        if len(scroll_response['hits']['hits']) == 0:
            break
        for hit in scroll_response['hits']['hits']:
            count += 1
    print(count)

    end = time.time()
    print(f"Retrieving all documents with search and scroll took {end - start} seconds")